# Lesson 18: Hunting Exoplanets In Space - Data Normalisation

### Teacher-Student Activities

In the last class, we evaluated the Random Forest Classifier model through a concept called the **confusion matrix**. We also calculated the precision, recall and f1-score values. They were undefined. Based on these three parameters, we concluded that our model needs a lot of improvement because it did not classify the stars having a planet as `2` rather it labelled every star as `1`.

In this lesson, we will process the data before deploying a prediction model so that it can learn the properties of the different stars through the training dataset.

Now, there is no right approach to the data processing method. It is an iterative process. It comes through experience and domain knowledge. (*The term 'domain' means a particular field of industry or academics*). For e.g., if you are a banker, then you would have the knowledge of the finance field. Similarly, if you are an astrophysicist, then you would have the technical knowledge of astronomy, quantum mechanics, optics etc. So, based on the knowledge of a respective field, data should be processed before deploying a prediction model.

In this class, we will perform the following data processing exercises:

1. Data Normalisation

2. Fast Fourier Transformation

3. Oversampling

Finally, we will check if the Random Forest Classifier model is still providing the expected results. If not, then we will deploy a much stronger prediction model called XGBoost Classifier. Generally, it doesn't require a lot of data pre-processing but it requires heavy computation resources such as high RAM, CPU, GPU and at least 4 cores processor. Thanks to Google Colab notebook, we have a decent computation power to run the XGBoost Classifier algorithm.

Hence, the XGBoost Classifier model must be used only when the data processing is not helping a prediction model in making accurate predictions.

Let's run all the codes in the code cells that we have already covered in the previous classes and begin this class from the **Activity 1: Data Normalisation** section. You too run the code cells until the first activity.


---

#### Loading The Datasets

Create a Pandas DataFrame every time you start the Jupyter notebook.

Dataset links (don't click on them):

1. Train dataset

   https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/kepler-exoplanets-dataset/exoTrain.csv

2. Test dataset

   https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/kepler-exoplanets-dataset/exoTest.csv

In [ ]:
# Load both the training and test datasets.
import numpy as np
import pandas as pd

exo_train_df = pd.read_csv('https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/kepler-exoplanets-dataset/exoTrain.csv')
exo_test_df = pd.read_csv('https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/whitehat-ds-datasets/kepler-exoplanets-dataset/exoTest.csv')

In [ ]:
# The shapes of the 'exo_train_df' and 'exo_test_df' DataFrames.
print(exo_train_df.shape)
exo_test_df.shape

(5087, 3198)


(570, 3198)

In the previous classes, we have already checked the datasets don't have a missing value. So, we can skip that part.

---

#### Activity 1: Data Normalisation^^^

After creating a DataFrame and inspecting data for the missing values, we can normalise the data.

**What is data normalisation?**

Data normalisation is a process of standardising data. It brings every single data-point on a uniform scale. If you look at both the `exo_train_df` and `exo_test_df` DataFrames, they contain highly varying `FLUX` values.

You can get the data summary using the `describe()` function to look at the variation of the data.

**Syntax:** `dataframe_name.describe()`

In [ ]:
# Teacher Action: Get the data description by calling the 'describe()' function.
exo_train_df.describe()

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
count,5087.000000,5.087000e+03,5.087000e+03,5.087000e+03,5.087000e+03,5.087000e+03,5.087000e+03,5.087000e+03,5.087000e+03,5.087000e+03,...,5.087000e+03,5.087000e+03,5.087000e+03,5.087000e+03,5.087000e+03,5.087000e+03,5087.000000,5087.000000,5087.000000,5087.000000
mean,1.007273,1.445054e+02,1.285778e+02,1.471348e+02,1.561512e+02,1.561477e+02,1.469646e+02,1.168380e+02,1.144983e+02,1.228639e+02,...,3.485578e+02,4.956476e+02,6.711211e+02,7.468790e+02,6.937372e+02,6.553031e+02,-494.784966,-544.594264,-440.239100,-300.536399
std,0.084982,2.150669e+04,2.179717e+04,2.191309e+04,2.223366e+04,2.308448e+04,2.410567e+04,2.414109e+04,2.290691e+04,2.102681e+04,...,2.864786e+04,3.551876e+04,4.349963e+04,4.981375e+04,5.087103e+04,5.339979e+04,17844.469520,17722.339334,16273.406292,14459.795577
min,1.000000,-2.278563e+05,-3.154408e+05,-2.840018e+05,-2.340069e+05,-4.231956e+05,-5.975521e+05,-6.724046e+05,-5.790136e+05,-3.973882e+05,...,-3.240480e+05,-3.045540e+05,-2.933140e+05,-2.838420e+05,-3.288214e+05,-5.028894e+05,-775322.000000,-732006.000000,-700992.000000,-643170.000000
25%,1.000000,-4.234000e+01,-3.952000e+01,-3.850500e+01,-3.505000e+01,-3.195500e+01,-3.338000e+01,-2.813000e+01,-2.784000e+01,-2.683500e+01,...,-1.760000e+01,-1.948500e+01,-1.757000e+01,-2.076000e+01,-2.226000e+01,-2.440500e+01,-26.760000,-24.065000,-21.135000,-19.820000
50%,1.000000,-7.100000e-01,-8.900000e-01,-7.400000e-01,-4.000000e-01,-6.100000e-01,-1.030000e+00,-8.700000e-01,-6.600000e-01,-5.600000e-01,...,2.600000e+00,2.680000e+00,3.050000e+00,3.590000e+00,3.230000e+00,3.500000e+00,-0.680000,0.360000,0.900000,1.430000
75%,1.000000,4.825500e+01,4.428500e+01,4.232500e+01,3.976500e+01,3.975000e+01,3.514000e+01,3.406000e+01,3.170000e+01,3.045500e+01,...,2.211000e+01,2.235000e+01,2.639500e+01,2.909000e+01,2.780000e+01,3.085500e+01,18.175000,18.770000,19.465000,20.280000
max,2.000000,1.439240e+06,1.453319e+06,1.468429e+06,1.495750e+06,1.510937e+06,1.508152e+06,1.465743e+06,1.416827e+06,1.342888e+06,...,1.779338e+06,2.379227e+06,2.992070e+06,3.434973e+06,3.481220e+06,3.616292e+06,288607.500000,215972.000000,207590.000000,211302.000000


As you can see, the values in the `FLUX.1` column range between `-227,856.3` (minimum `FLUX.1` value) and `1,439,240` (maximum `FLUX.1` value).

**Note:** `-2.278563e+05` is equivalent to $-2.278563\times10^5$ and `1.439240e+06` equivalent to $1.439240\times10^6$

In the `FLUX.1` column, the difference in the maximum and minimum values, i.e.,

$$1,439,240 - (-227,856.3) = 1,667,096.3$$


is huge because of the $10^5$ and $10^6$ scales.

Similarly, the data in all the other `FLUX` columns also vary a lot because they lie on a huge scale. The big figures are less readable. For e.g., `122` (one hundred twenty two) is more readable than `1,439,240` (one million, four hundred thirty nine thousand, two hundred forty).

The data normalisation process lowers the scale and brings all the data-points on the same scale.

**Why must data be normalised?**

The machine learning models are quite sensitive to the scale of data. They give more importance to the larger values while learning the properties of data. Hence, it becomes crucial for us to remove this bias by bringing down all the data-points on the same scale.


**How to normalise data?**

There are various methods of data normalisation. We will cover all of them throughout this course whenever we need them. For the time being, we will use the *mean normalisation* method. Let's understand the *mean normalisation* method.

Consider a series of numbers having the values

$$x_1, x_2, x_3, ... , x_N$$

where $N$ is the total number of values in a series.

Let

- $x_{mean}$ denote the mean (or average) value of a series

- $x_{min}$ denote the minimum value in a series and

- $x_{max}$ denote the maximum value in a series

The normalised value in a series is calculated as

$$x_{norm} = \frac{x_p - x_{mean}}{x_{max} - x_{min}}$$

where

$$x_p = x_1, x_2, x_3, ..., x_N$$

So after normalisation, the new values in the series will be

$$\left(\frac{x_1 - x_{mean}}{x_{max} - x_{min}}\right), \left(\frac{x_2 - x_{mean}}{x_{max} - x_{min}}\right), \left(\frac{x_3 - x_{mean}}{x_{max} - x_{min}}\right), ..., \left(\frac{x_N - x_{mean}}{x_{max} - x_{min}}\right)$$

For e.g., consider the series $[5, 192, 20019, 12, 209]$.

- The average value of the series is $x_{mean} = 4087.4$

- The minimum value in the series is $x_{min} = 5$

- The maximum value in the series is $x_{max} = 20019$

So, after normalisation, the new series would have the following numbers.

$$\left[ \left( \frac{5 - 4087.4}{20019 - 5} \right), \left( \frac{192 - 4087.4}{20019 - 5} \right), \left( \frac{20019 - 4087.4}{20019 - 5} \right), \left( \frac{12 - 4087.4}{20019 - 5} \right), \left( \frac{209 - 4087.4}{20019 - 5} \right) \right]$$

$$\Rightarrow \left[-0.203977, -0.194634, 0.796023, -0.203627, -0.193784 \right]$$

or

$$\left[-\frac{203,977}{1,000,000}, -\frac{194,634}{1,000,000}, \frac{796,023}{1,000,000}, -\frac{203,627}{1,000,000}, -
\frac{193,784}{1,000,000} \right]$$

As you can see, after normalisation all the new values are on the same scale which is $\frac{1}{1,000,000}$ or $10^{-6}$.

So, now let's create a function which normalises data in a series. It should take a Pandas series as an input and should return a normalised series as an output.





In [ ]:
# Student Action: Create a function to normalise a Pandas series using the mean normalisation method.
def mean_normalise(ser):
  mean = ser.mean()
  min = ser.min()
  max = ser.max()
  nor_ser = (ser - mean) / (max - min)
  return nor_ser


In [ ]:
def norm(ser):
  return ((ser - ser.mean())/(ser.max() - ser.min()))

In [ ]:
norm(pd.Series([5, 192, 20019, 12, 209]))

0   -0.203977
1   -0.194634
2    0.796023
3   -0.203627
4   -0.193784
dtype: float64

Now, let's test the `mean_normalise()` function on the $[5, 192, 20019, 12, 209]$ series. If we get the desired output, then it means the function is working correctly.

In [ ]:
# Student Action: Test the 'mean_normalise()' function on the '[5, 192, 20019, 12, 209]' series.
mean_normalise(pd.Series([5, 192, 20019, 12, 209]))

0   -0.203977
1   -0.194634
2    0.796023
3   -0.203627
4   -0.193784
dtype: float64

Now, let's apply the `mean_normalise()` function on the `exo_train_df` DataFrame to normalise only the `FLUX` values.

Using the `iloc[]` function, we will first exclude the `LABEL` column from the DataFrame and then will apply the `mean_normalise()` function on the `exo_train_df` DataFrame using the `apply()` function.

The `apply()` function takes two inputs.

- The first input is the function that needs to be applied which in this case is `mean_normalise()` function.

- The second input is the `axis` value which signifies whether the function in the first input needs to be applied vertically or horizontally.

**Syntax:** `dataframe.apply(function_name, axis)`

**Note:** Whenever you apply a function, say `function_name()` on a DataFrame using the `apply()` function, remove the brackets from the name of the function (i.e., `function_name`) to be applied. That's why the syntax is `dataframe.apply(function_name, axis)`

A DataFrame has two axes (axes is plural of axis).

- The first axis is the vertical axis. It is represented as `axis = 0`

- The second axis is the horizontal axis. It is represented as `axis = 1`

The DataFrame axes define whether an operation needs to be applied row-wise or column-wise. Refer to the image shown below.

<img src='https://student-datasets-bucket.s3.ap-south-1.amazonaws.com/images/lesson-16/dataframe-axes.png' width=600>

1. If `axis = 0`, then it means the function needs to be applied **vertically**. In other words, the function will be applied on all the rows but only **one column at a time.** So, on the `exo_train_df` DataFrame, if the `mean_normalise()` function is applied **vertically**, then it will be applied in the following order:

    - `exo_train_df.iloc[:, 0]`, i.e., all the rows and the `FLUX.1` column at a time.
    
    - `exo_train_df.iloc[:, 1]`, i.e., all the rows and the `FLUX.2` column at a time.

    - `exo_train_df.iloc[:, 2]`, i.e., all the rows and the `FLUX.3` column at a time.

    ...

    - `exo_train_df.iloc[:, 3197]`, i.e., all the rows and the `FLUX.3197` column at a time.

2. If `axis = 1`, then it means the function needs to be applied **horizontally**. This means the function will be applied on all the columns but only **one row at a time.** So, on the `exo_train_df` DataFrame, if the `mean_normalise()` function is applied **horizontally**, then it will be applied in the following order:

    - `exo_train_df.iloc[0, :]`, i.e., the first row and all the columns at a time.
    
    - `exo_train_df.iloc[1, :]`, i.e., the second row and all the columns at a time.

    - `exo_train_df.iloc[2, :]`, i.e., the third row and all the columns at a time.

      ...

    - `exo_train_df.iloc[5086, :]`, i.e., the last row and all the columns at a time.

We will apply the `mean_normalise()` function **horizontally** on the `exo_train_df` DataFrame to normalise the `FLUX` values for a star at a time.


In [ ]:
# Teacher Action: Apply the 'mean_normalise' function horizontally on the training DataFrame.
nom_train_fd = exo_train_df.iloc[:,1:].apply(mean_normalise, axis = 1)
# After applying the 'mean_normalise' function on the 'exo_train_df' DataFrame, let's print the first 5 rows of the new DataFrame.
nom_train_fd

,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,0.053834,0.047391,0.006510,-0.023699,-0.031772,-0.086410,-0.093128,-0.068161,-0.057650,-0.109164,...,-0.056482,-0.071934,-0.071934,0.009738,0.024779,0.052993,0.018843,0.033024,-0.003127,-0.031759
1,-0.050411,-0.042317,-0.081922,-0.052351,-0.115212,-0.104794,-0.126816,-0.124861,-0.122681,-0.105708,...,0.006648,-0.039721,-0.039721,-0.027988,0.004116,0.013124,-0.006847,0.022260,0.037550,0.043849
2,0.243983,0.245509,0.235186,0.227365,0.208538,0.212981,0.212283,0.222467,0.199285,0.221536,...,-0.037161,0.002382,0.002382,-0.017715,-0.013523,-0.001456,-0.009299,-0.017259,-0.036384,-0.048782
3,0.518501,0.551177,0.480659,0.474051,0.504754,0.496863,0.511941,0.494687,0.496425,0.513506,...,0.016215,0.001435,0.001435,0.054324,0.038636,-0.012562,-0.006456,-0.019827,-0.019889,0.029163
4,-0.399904,-0.401872,-0.404199,-0.395473,-0.381734,-0.373293,-0.360070,-0.368986,-0.356861,-0.350022,...,-0.212262,-0.141752,-0.141752,-0.125499,-0.157156,-0.155246,-0.141038,-0.135528,-0.145458,-0.181590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,-0.191318,-0.193261,-0.167219,-0.201251,-0.147500,-0.147940,-0.161208,-0.113139,-0.079547,-0.044654,...,0.233591,0.246987,0.264745,0.262344,0.263664,0.255216,-0.067690,-0.054037,-0.016964,0.012450
5083,0.419089,0.376920,0.384402,0.360187,0.318881,0.259377,0.250831,0.192365,0.206873,0.160536,...,-0.014922,-0.005672,-0.036191,-0.019765,-0.069301,-0.061755,0.012637,0.039222,-0.023097,0.011590
5084,0.132006,0.134256,0.126314,0.114239,0.135588,0.127862,0.122014,0.122971,0.114537,0.114981,...,-0.014526,-0.027738,-0.025210,0.013691,0.006355,-0.003129,0.030648,0.041723,0.037159,0.037335
5085,0.005570,0.002376,-0.007557,-0.006800,0.001582,-0.002867,0.005625,-0.001538,0.004573,0.010130,...,0.018567,-0.007446,-0.010899,-0.009994,-0.019613,-0.004252,-0.010068,-0.028346,-0.013317,-0.006191


In [ ]:
nom_train_fd.describe()

,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
count,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,...,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000,5087.000000
mean,0.000685,-0.000596,-0.001566,-0.001202,-0.000740,-0.002470,-0.000730,-0.001281,-0.001421,-0.002037,...,0.005165,0.004794,0.006335,0.008774,0.006500,0.008134,-0.005690,-0.003288,-0.001603,-0.001082
std,0.168811,0.163252,0.158553,0.153144,0.147773,0.145188,0.140170,0.135593,0.130129,0.127598,...,0.093178,0.094372,0.096610,0.103292,0.105772,0.110695,0.096611,0.094191,0.091258,0.091374
min,-0.779681,-0.747867,-0.747912,-0.778677,-0.745649,-0.740931,-0.846553,-0.714771,-0.640588,-0.667493,...,-0.608219,-0.753987,-0.639173,-0.648696,-0.661599,-0.668388,-0.825602,-0.631304,-0.592546,-0.597935
25%,-0.063742,-0.063744,-0.062319,-0.057512,-0.052546,-0.054312,-0.049331,-0.046963,-0.045058,-0.045002,...,-0.024968,-0.026192,-0.025883,-0.026775,-0.030718,-0.031387,-0.037260,-0.032210,-0.030542,-0.029569
50%,-0.002517,-0.002839,-0.002772,-0.002135,-0.002052,-0.002845,-0.002978,-0.002874,-0.002454,-0.003042,...,0.004009,0.003962,0.004231,0.005519,0.005356,0.005589,-0.002089,-0.000566,0.000496,0.001265
75%,0.060475,0.057469,0.054974,0.052705,0.048321,0.044719,0.044202,0.041961,0.038599,0.038929,...,0.036174,0.037022,0.039469,0.046060,0.046883,0.051388,0.027713,0.028485,0.030228,0.031811
max,0.927562,0.909774,0.975016,0.892899,0.781205,0.913806,0.916485,0.819859,0.805654,0.865495,...,0.876997,0.629574,0.656747,0.643289,0.980652,0.648446,0.915423,0.958154,0.653278,0.570174


In [ ]:
nom_train_fd.head(3)

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,0.053834,0.047391,0.006510,-0.023699,-0.031772,-0.086410,-0.093128,-0.068161,-0.057650,...,-0.056482,-0.071934,-0.071934,0.009738,0.024779,0.052993,0.018843,0.033024,-0.003127,-0.031759
1,2,-0.050411,-0.042317,-0.081922,-0.052351,-0.115212,-0.104794,-0.126816,-0.124861,-0.122681,...,0.006648,-0.039721,-0.039721,-0.027988,0.004116,0.013124,-0.006847,0.022260,0.037550,0.043849
2,2,0.243983,0.245509,0.235186,0.227365,0.208538,0.212981,0.212283,0.222467,0.199285,...,-0.037161,0.002382,0.002382,-0.017715,-0.013523,-0.001456,-0.009299,-0.017259,-0.036384,-0.048782


In [ ]:
round(nom_train_fd.iloc[1,1:].mean())

0

In [ ]:
round(nom_train_fd.iloc[:,1:].mean())

FLUX.1       0.0
FLUX.2      -0.0
FLUX.3      -0.0
FLUX.4      -0.0
FLUX.5      -0.0
            ... 
FLUX.3193    0.0
FLUX.3194   -0.0
FLUX.3195   -0.0
FLUX.3196   -0.0
FLUX.3197   -0.0
Length: 3197, dtype: float64

You can see that, all the data-points are on the same scale after mean normalisation. Notice that we didn't normalise the `LABEL` data as we intended.

Now, let's insert the `LABEL` column to the `norm_train_df` DataFrame to get the full DataFrame with the normalised `FLUX` values.

We can obtain the `LABEL` column from the `exo_train_df` DataFrame using the `exo_train_df['LABEL']` method.

To insert a column in a DataFrame, use the `insert()` function. It takes three inputs.

- The first input should be the desired column index of the new column after its insertion.

- The second input should be the desired column name.

- The third input should be the values of the new column.

**Syntax:** `dataframe.insert(loc=column_index, column=column_name, value=some_pandas_series)`

In [ ]:
# Teacher Action: Apply the 'insert()' function to add the 'LABEL' column to the 'norm_train_df' DataFrame.
nom_train_fd.insert(loc = 0, column = 'LABEL', value=exo_train_df['LABEL'])
# After inserting the 'LABEL' column to the 'norm_train_df' DataFrame, print its first five rows.
nom_train_fd

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,0.053834,0.047391,0.006510,-0.023699,-0.031772,-0.086410,-0.093128,-0.068161,-0.057650,...,-0.056482,-0.071934,-0.071934,0.009738,0.024779,0.052993,0.018843,0.033024,-0.003127,-0.031759
1,2,-0.050411,-0.042317,-0.081922,-0.052351,-0.115212,-0.104794,-0.126816,-0.124861,-0.122681,...,0.006648,-0.039721,-0.039721,-0.027988,0.004116,0.013124,-0.006847,0.022260,0.037550,0.043849
2,2,0.243983,0.245509,0.235186,0.227365,0.208538,0.212981,0.212283,0.222467,0.199285,...,-0.037161,0.002382,0.002382,-0.017715,-0.013523,-0.001456,-0.009299,-0.017259,-0.036384,-0.048782
3,2,0.518501,0.551177,0.480659,0.474051,0.504754,0.496863,0.511941,0.494687,0.496425,...,0.016215,0.001435,0.001435,0.054324,0.038636,-0.012562,-0.006456,-0.019827,-0.019889,0.029163
4,2,-0.399904,-0.401872,-0.404199,-0.395473,-0.381734,-0.373293,-0.360070,-0.368986,-0.356861,...,-0.212262,-0.141752,-0.141752,-0.125499,-0.157156,-0.155246,-0.141038,-0.135528,-0.145458,-0.181590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,1,-0.191318,-0.193261,-0.167219,-0.201251,-0.147500,-0.147940,-0.161208,-0.113139,-0.079547,...,0.233591,0.246987,0.264745,0.262344,0.263664,0.255216,-0.067690,-0.054037,-0.016964,0.012450
5083,1,0.419089,0.376920,0.384402,0.360187,0.318881,0.259377,0.250831,0.192365,0.206873,...,-0.014922,-0.005672,-0.036191,-0.019765,-0.069301,-0.061755,0.012637,0.039222,-0.023097,0.011590
5084,1,0.132006,0.134256,0.126314,0.114239,0.135588,0.127862,0.122014,0.122971,0.114537,...,-0.014526,-0.027738,-0.025210,0.013691,0.006355,-0.003129,0.030648,0.041723,0.037159,0.037335
5085,1,0.005570,0.002376,-0.007557,-0.006800,0.001582,-0.002867,0.005625,-0.001538,0.004573,...,0.018567,-0.007446,-0.010899,-0.009994,-0.019613,-0.004252,-0.010068,-0.028346,-0.013317,-0.006191


Now, you normalise the `FLUX` values in the `exo_test_df` DataFrame using the `mean_normalise()` function. Make sure that you apply the function horizontally to normalise the `FLUX` values for a star at a time.

In [ ]:
# Student Action: Apply the 'mean_normalize()' function on the testing DataFrame. Store the new DataFrame in the 'norm_test_df' variable.
nom_test_df = exo_test_df.iloc[:,1:].apply(mean_normalise, axis = 1)
# After applying, the 'mean_normalise()' function on the 'exo_test_df' DataFrame, print the first 5 rows of the new DataFrame.
nom_test_df

,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,FLUX.10,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,0.273347,0.228221,0.196676,0.110003,0.104130,0.088690,0.040926,0.014337,0.013534,-0.052079,...,0.031635,0.042578,0.031451,-0.005393,0.028904,0.102708,0.071576,0.080408,0.616438,0.130742
1,0.394038,0.391480,0.392680,0.390974,0.388955,0.386673,0.386340,0.382364,0.381035,0.374634,...,-0.047311,-0.075404,-0.092643,-0.118456,-0.134109,-0.150638,-0.164944,-0.171944,-0.166961,-0.148790
2,0.648150,0.627582,0.591444,0.519002,0.466046,0.385214,0.340496,0.281192,0.162553,0.119260,...,0.018179,-0.034769,-0.032201,-0.041117,-0.057967,-0.128412,-0.067972,-0.119374,-0.023437,0.027941
3,-0.232813,-0.233212,-0.238944,-0.235869,-0.208281,-0.220224,-0.222214,-0.208586,-0.197319,-0.188186,...,0.056186,0.047254,0.047254,0.039873,0.021893,0.025227,0.025075,-0.017912,-0.059585,-0.046740
4,-0.006994,0.003426,0.006382,0.007610,0.003316,-0.000167,0.010016,-0.009471,0.008195,0.016842,...,-0.006247,-0.016795,-0.001531,0.001095,-0.004439,-0.027127,-0.025421,-0.016852,-0.020089,0.002564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,0.129222,0.112160,0.109978,0.116450,0.085890,0.071190,0.062910,0.056251,0.044966,0.035036,...,-0.046336,-0.061618,-0.081197,-0.076709,-0.066292,-0.061593,-0.085857,-0.078091,-0.075398,-0.071196
566,0.002427,0.012929,0.015475,0.011428,0.019443,-0.015675,-0.016800,-0.005824,-0.019662,-0.017550,...,-0.021340,-0.008687,-0.032276,-0.020472,-0.020768,-0.023867,-0.003949,-0.007364,-0.018537,-0.019050
567,-0.192333,-0.158094,-0.148044,-0.153554,-0.141945,-0.091384,-0.113078,-0.090794,-0.050629,-0.056382,...,-0.007692,-0.010846,0.000313,-0.021866,-0.046506,-0.064561,-0.019891,0.013794,0.044949,0.059642
568,0.018086,0.017085,0.010689,0.010668,0.014381,0.006080,0.004226,0.008785,0.007118,0.001446,...,0.002628,0.004208,-0.000651,-0.000901,0.005872,0.000422,-0.004239,0.000738,0.001077,0.005263


Now, you insert the `LABEL` column to the `norm_test_df` DataFrame with the corresponding `LABEL` values.

In [ ]:
# Student Action: Apply the 'insert()' function to add the 'LABEL' column to the 'norm_test_df' DataFrame.
nom_test_df.insert(loc = 0, column = 'LABEL', value= exo_test_df['LABEL'])
# After inserting the 'LABEL' column to the 'norm_test_df' DataFrame, print its first five rows.
nom_test_df

,LABEL,FLUX.1,FLUX.2,FLUX.3,FLUX.4,FLUX.5,FLUX.6,FLUX.7,FLUX.8,FLUX.9,...,FLUX.3188,FLUX.3189,FLUX.3190,FLUX.3191,FLUX.3192,FLUX.3193,FLUX.3194,FLUX.3195,FLUX.3196,FLUX.3197
0,2,0.273347,0.228221,0.196676,0.110003,0.104130,0.088690,0.040926,0.014337,0.013534,...,0.031635,0.042578,0.031451,-0.005393,0.028904,0.102708,0.071576,0.080408,0.616438,0.130742
1,2,0.394038,0.391480,0.392680,0.390974,0.388955,0.386673,0.386340,0.382364,0.381035,...,-0.047311,-0.075404,-0.092643,-0.118456,-0.134109,-0.150638,-0.164944,-0.171944,-0.166961,-0.148790
2,2,0.648150,0.627582,0.591444,0.519002,0.466046,0.385214,0.340496,0.281192,0.162553,...,0.018179,-0.034769,-0.032201,-0.041117,-0.057967,-0.128412,-0.067972,-0.119374,-0.023437,0.027941
3,2,-0.232813,-0.233212,-0.238944,-0.235869,-0.208281,-0.220224,-0.222214,-0.208586,-0.197319,...,0.056186,0.047254,0.047254,0.039873,0.021893,0.025227,0.025075,-0.017912,-0.059585,-0.046740
4,2,-0.006994,0.003426,0.006382,0.007610,0.003316,-0.000167,0.010016,-0.009471,0.008195,...,-0.006247,-0.016795,-0.001531,0.001095,-0.004439,-0.027127,-0.025421,-0.016852,-0.020089,0.002564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,1,0.129222,0.112160,0.109978,0.116450,0.085890,0.071190,0.062910,0.056251,0.044966,...,-0.046336,-0.061618,-0.081197,-0.076709,-0.066292,-0.061593,-0.085857,-0.078091,-0.075398,-0.071196
566,1,0.002427,0.012929,0.015475,0.011428,0.019443,-0.015675,-0.016800,-0.005824,-0.019662,...,-0.021340,-0.008687,-0.032276,-0.020472,-0.020768,-0.023867,-0.003949,-0.007364,-0.018537,-0.019050
567,1,-0.192333,-0.158094,-0.148044,-0.153554,-0.141945,-0.091384,-0.113078,-0.090794,-0.050629,...,-0.007692,-0.010846,0.000313,-0.021866,-0.046506,-0.064561,-0.019891,0.013794,0.044949,0.059642
568,1,0.018086,0.017085,0.010689,0.010668,0.014381,0.006080,0.004226,0.008785,0.007118,...,0.002628,0.004208,-0.000651,-0.000901,0.005872,0.000422,-0.004239,0.000738,0.001077,0.005263


---

#### Activity 2: Transpose Of A DataFrame^

After the data normalisation, we need to apply Fast Fourier Transformation (which we will learn in the next class). To apply Fast Fourier Transformation, we need to follow three steps.

1. Interchange rows and columns with each other so that columns become rows and rows become columns.

2. Apply the `fft()` function on the DataFrame to apply Fast Fourier Transformation. This we will study in the next class.

3. Interchange rows and columns again.

So, now we will first learn how to interchange the rows and columns. The process of interchanging rows and columns is called **transpose**. It's a very simple process.

To transpose a DataFrame, you can use the `T` keyword.

In [ ]:
# Student Action: Transpose the 'exo_train_df' using the 'T' keyword.
exo_train_df.T

,0,1,2,3,4,5,6,7,8,9,...,5077,5078,5079,5080,5081,5082,5083,5084,5085,5086
LABEL,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
FLUX.1,93.85,-38.88,532.64,326.52,-1107.21,211.10,9.34,238.77,-103.54,-265.91,...,125.57,7.45,475.61,-46.63,299.41,-91.91,989.75,273.39,3.82,323.28
FLUX.2,83.81,-33.83,535.92,347.39,-1112.59,163.57,49.96,262.16,-118.97,-318.59,...,78.69,10.02,395.50,-55.39,302.77,-92.97,891.01,278.00,2.09,306.36
FLUX.3,20.10,-58.54,513.73,302.35,-1118.95,179.16,33.30,277.80,-108.93,-335.66,...,98.29,6.87,423.61,-64.88,278.68,-78.76,908.53,261.73,-3.29,293.16
FLUX.4,-26.98,-40.09,496.92,298.13,-1095.10,187.82,9.63,190.16,-72.25,-450.47,...,91.16,-2.82,376.36,-88.75,263.48,-97.33,851.83,236.99,-2.88,287.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FLUX.3193,92.54,0.76,5.06,-12.67,-438.54,19.27,-0.44,95.30,4.53,3162.53,...,210.09,8.75,163.02,28.82,-74.95,151.75,-136.16,-3.47,-1.50,-25.33
FLUX.3194,39.32,-11.70,-11.80,-8.77,-399.71,-43.90,10.90,48.86,21.95,3398.28,...,3.80,-10.69,86.29,-20.12,-46.29,-24.45,38.03,65.73,-4.65,-41.31
FLUX.3195,61.42,6.46,-28.91,-17.31,-384.65,-41.63,-11.77,-10.62,26.94,3648.34,...,16.33,-9.54,13.06,-14.41,-3.08,-17.00,100.28,88.42,-14.55,-16.72
FLUX.3196,5.08,16.00,-70.02,-17.35,-411.79,-52.90,-9.25,-112.02,34.08,3671.97,...,27.35,-2.48,161.22,-43.35,-28.43,3.23,-45.64,79.07,-6.41,-14.09


As you can see, the rows have become columns and columns have become rows.

In the next class, we will apply Fast Fourier Transformation on the DataFrame.

---